### surfing_package 설치 및 구동

##### 1. 디렉토리 생성 및 package 구성

In [1]:
!rm -rf surfing

In [2]:
!mkdir -p surfing/surfing
!mkdir -p surfing/surfing/wsbfarm
!mkdir -p surfing/surfing/surfx
!touch surfing/setup.py
!touch surfing/surfing/__init__.py
!touch surfing/surfing/wsbfarm/__init__.py
!touch surfing/surfing/surfx/__init__.py

In [3]:
!tree surfing

surfing
├── setup.py
└── surfing
    ├── __init__.py
    ├── surfx
    │   └── __init__.py
    └── wsbfarm
        └── __init__.py

3 directories, 4 files


##### wsbfarm 모듈 구성

In [4]:
%%writefile surfing/surfing/wsbfarm/Wsbfarm.py


import requests
import os
import pandas as pd
import time
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent

class Wsbfarm:
    '''
    서비스 설명 : WsbFarm으로부터 파도 데이터들을 가지고 오는 서비스입니다.
    객체로 받아 클래스를 실행하세요. ex) wsb = Wsbfarm()
    
    - beach_list() : WsbFarm에 등록된 해변 리스트들을 불러오는 서비스입니다.
    - login(email, pw): WsbFarm에 로그인 하는 메소드입니다. wsbfarm id, pw를 입력하세요
    - get_wave_beach('원하는 해변 코드') : 원하는 해변의 10일치 파도 정보를 가지고 오는 서비스입니다.
    - get_wave_data() : 모든 해변 (30개)의 10일치 파도 정보를 가지고 오는 서비스입니다. 불러온 데이터는 'wsbfarm.csv' 파일로 저장됩니다.
    - get_wave_date(yyyymmdd) : 원하는 날짜의 해변들의 파도 정보를 가지고 오는 서비스입니다. (ex, 20200930)
    - close() : 서비스를 종료하는 기능입니다.
    '''
    
    # 생성자 함수
    def __init__(self):
        wsb_url = "https://www.wsbfarm.com/wavecam/WaveCamList#video_list"        
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent={}".format(UserAgent().chrome))
        options.add_argument("headless")
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(1)
        self.driver.get(wsb_url) 
        self.result_df = None
        self.yyyymmdd = None
    
    # 해변 리스트 출력 메서드 : 중복 해변 제외한 30개 해변
    def beach_list(self):
        '''
        WsbFarm에 등록된 해변 리스트들을 불러오는 서비스입니다.
        '''
        
        url = "https://www.wsbfarm.com/wavecam/WaveCamList"
        self.driver.get(url)
        
        beach_lists = self.driver.find_elements_by_css_selector('#wrapper > div.index.main_content > div.main_content_inner.camList > div.swiper-container.gallery-top.swiper-container-initialized.swiper-container-horizontal.swiper-container-autoheight > ul > li.swiper-slide.swiper-slide-active > div > div > a')

        beaches = []

        for beach_list in beach_lists[:41]:
            beach_eng = beach_list.get_attribute('data-urlid')
            beach_kor = beach_list.find_element_by_css_selector('a > div.video-post-content.st01 > h4').text
            beaches.append({'kor':beach_kor, 'eng':beach_eng})
        
        beach_list_df = pd.DataFrame(beaches)
        self.beach_list_df = beach_list_df
        self.beach_list_df = beach_list_df.loc[~beach_list_df['kor'].str.contains('B') & ~beach_list_df['kor'].str.contains('C')]
        self.beach_list_df.reset_index(inplace=True, drop=True)
        return self.beach_list_df
    
    # 로그인
    def login(self, email, pw):
        '''
        WsbFarm에 로그인 하는 메소드입니다.
        login(wsbfarm email, wsbfarm pw) 와 같이 입력하세요.
        '''
        
        self.email = email
        self.pw = pw
        self.driver.find_element_by_css_selector('#main_header > header > div.head_user > button:nth-child(4)').click()
        time.sleep(3)
        self.driver.find_element_by_xpath('//*[@id="userEmail"]').send_keys(email)
        time.sleep(1)
        self.driver.find_element_by_xpath('//*[@id="password"]').send_keys(pw)
        time.sleep(1)
        self.driver.find_element_by_xpath('//*[@id="wrapper"]/div[3]/div/div/form/fieldset/div[1]/button').click()
        try:
            WebDriverWait(self.driver, 3).until(EC.alert_is_present())
            alert = self.driver.switch_to.alert
            alert.accept()
            print("Login Error!!!")
            time.sleep(1)
        except:
            print("Login Sucessful!!!")
        
    # 특정 해변 상세정보 크롤링
    def get_wave_beach(self, beach_eng):
        '''
        특정 해변의 상세정보를 크롤링하는 메소드입니다. 원하는 해변의 파도가 가장 높을 때의 시간, 바람, 온도 등을 표시합니다.
        get_wave_beach(원하는 해변의 코드) 를 입력하세요.
        '''
        
        self.beach_eng = beach_eng
        url = "https://www.wsbfarm.com/wavecam/WaveCamView?beach={}".format(beach_eng)
        self.driver.get(url)
        self.driver.implicitly_wait(1)
        # 파도 정보는 오늘 기준 10일치 데이터만 제공됨
        # y좌표값 가져오기
        yaxis = []
        wave_data = []
        i = 0
        try:
            while i < 10:
                today = datetime.date.today()
                sdate = today + datetime.timedelta(days = i)
                sdate = sdate.strftime('%Y%m%d')
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 로딩을 위해 우선 스크롤 내려주고
                axis = self.driver.find_element_by_css_selector('.beachData[data-date="{}"'.format(sdate)).location
                yaxis.append({'date':sdate, 'location':axis['y']})
                for j in range(1, 7):
                    time_str = self.driver.find_element_by_css_selector('.beachData[data-date="{}"] > table > tbody > tr:nth-child({}) > td:nth-child({})'.format(sdate, j, 1)).text
                    wind = self.driver.find_element_by_css_selector('.beachData[data-date="{}"] > table > tbody > tr:nth-child({}) > td:nth-child({})'.format(sdate, j, 2)).text
                    temp = self.driver.find_element_by_css_selector('.beachData[data-date="{}"] > table > tbody > tr:nth-child({}) > td:nth-child({})'.format(sdate, j, 3)).text.split('o')[0]
                    wave = self.driver.find_element_by_css_selector('.beachData[data-date="{}"] > table > tbody > tr:nth-child({}) > td:nth-child({})'.format(sdate, j, 4)).text.replace('\n', ' & ').split('m')[0]
                    wave_tide = self.driver.find_element_by_css_selector('.beachData[data-date="{}"] > table > tbody > tr:nth-child({}) > td:nth-child({})'.format(sdate, j, 5)).text.split('m')[0]
                    wave_data.append({"date": sdate, "time": time_str, "wind": wind, "temp": temp, "wave": float(wave), "wave_tide": float(wave_tide)})
                i += 1
                wave_df = pd.DataFrame(wave_data)
                wave_df = wave_df[(wave_df['time'] != '오후 09') & (wave_df['time'] != '오전 12') & (wave_df['time'] != '오전 03')]
                wave_df = wave_df.sort_values(['date', 'wave'])
                wave_df = wave_df.groupby('date').tail(1)
            # 위에서 구한 y좌표값으로 10일치 파도 요약정보 불러오기
            wave_summary = []
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            for i in range(10):
                self.driver.execute_script("window.scrollTo(0,{});".format(yaxis[i]['location']))
                time.sleep(0.5)
                self.driver.execute_script("window.scrollTo(0,{});".format(yaxis[i]['location']-10))
                time.sleep(0.5)
                summary = self.driver.find_element_by_css_selector('#con1 > div.beachToday.p_todayWaveinfo.fix.p_{} > div.wave'.format(yaxis[i]['date'])).text.split('오늘의 파도')[1].strip().replace("\n", ". ")
                time.sleep(1)
                wave_summary.append({'eng':beach_eng, 'date':yaxis[i]['date'], 'summary':summary})
            summary_df = pd.DataFrame(wave_summary)
            # 두개 데이터프레임 merge 시키기
            self.result_df = summary_df.merge(wave_df, on='date')
            return self.result_df
        except:
            pass
    
    # 모든 해변에 대한 정보 가져오기
    def get_wave_data(self):
        '''
        모든 해변에 대한 정보들을 가지고 오는 메소드입니다.
        get_wave_data() 를 실행하세요.
        '''
        dfs = []            
        for beach_eng in self.beach_list_df['eng']:
            print("beach:{} crawling ...".format(beach_eng))
            self.beach_eng = beach_eng
            df = self.get_wave_beach(beach_eng)
            dfs.append(df)
        self.result_df = pd.concat(dfs, ignore_index=True, sort=False)
        
        if not os.path.isfile('wsbfarm.csv'): # if file does not exist 
            self.result_df.to_csv('wsbfarm.csv', index=False, encoding='utf-8')
        else: # else it exists so append without writing the header
            self.result_df.to_csv('wsbfarm.csv', mode='a', header=False, index=False, encoding='utf-8')
        print("WSBfarm CSV saved!")
        
    # 특정 날짜에 대한 파도 데이터 보기
    def get_wave_date(self, yyyymmdd):
        '''
        특정 날짜에 대한 해변들의 파도 데이터를 가지고 오는 메소드입니다.
        get_wave_date(20200920)와 같이 입력하세요.
        '''
        self.yyyymmdd = yyyymmdd
        today = datetime.date.today()
        crawling_date = today + datetime.timedelta(days = 10)
        if int(yyyymmdd) < int(crawling_date.strftime('%Y%m%d')): # 파도 데이터는 10일치만 제공함 
            # 해변, 날짜, 시간대가 같은 경우 중복으로 여겨 가장 최근 정보만 남기고 drop
            df = pd.read_csv('wsbfarm.csv')
            df.drop_duplicates(subset=['eng', 'date', 'time'], keep='last', inplace=True) 
            df.reset_index(inplace=True, drop=True)
            # 데이터프레임 정리해주기
            self.result_df = df[df['date'] == yyyymmdd]
            self.result_df.reset_index(inplace=True, drop=True)
            self.result_df = self.result_df.merge(self.beach_list_df, on='eng')
            self.result_df.drop(columns=['date', 'eng'], inplace=True)
            self.result_df.rename(columns={"kor":"beach", "wave":"max_wave"}, inplace=True)
            self.result_df = self.result_df[['beach', 'summary', "time", "wind", "temp", "max_wave", "wave_tide"]]
            self.result_df.sort_values(by=['max_wave', 'beach'], ascending=False, inplace=True) # 파도가 높은순으로 정렬
            self.result_df.reset_index(inplace=True, drop=True)
            return self.result_df
        else:
            print("파도 데이터는 {}까지 데이터만 제공합니다. 다른 날짜를 입력해주세요".format((today + datetime.timedelta(days = 9)).strftime('%Y%m%d')))
    
    # 종료
    def close(self):
        '''
        프로그램을 종료하는 메소드입니다.
        wsb.close()를 입력하세요.
        '''
        self.driver.close()

Writing surfing/surfing/wsbfarm/Wsbfarm.py


##### surfx 모듈 구성

In [5]:
%%writefile surfing/surfing/surfx/Surfx.py

import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import os
import re
import datetime
import random

class Surfx:
    # 생성자
    def __init__(self):
        url = 'https://m.cafe.naver.com/surfx'
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(3)
        self.driver.get(url)
    
    # 로그인
    def login(self, id, pw):
        self.id = id
        self.pw = pw
        self.driver.find_element_by_xpath('//*[@id="hd"]/div/div[4]/a').click() # 사이드 메뉴 클릭
        self.driver.find_element_by_xpath('//*[@id="sideMenuContainer"]/header/div/div/div/a[2]').click() # 로그인 클릭
        input_js = 'document.getElementById("id").value = "{}";document.getElementById("pw").value = "{}";'.format(self.id, self.pw)
        time.sleep(random.uniform(1,3)) # 자동화탐지를 우회 하기 위한 delay
        self.driver.execute_script(input_js)
        time.sleep(random.uniform(1,3)) # 자동화탐지를 우회 하기 위한 delay
        self.driver.find_element_by_xpath('//*[@id="log.login"]').click() # 로그인 버튼 클릭
        print("Done")
        
    def __get_list(self):  
        self.driver.find_element_by_xpath('//*[@id="hd"]/div/div[4]/a').click() # 사이드 메뉴 클릭
        self.driver.find_element_by_xpath('//*[@id="sideMenuContainer"]/div/div[2]/div[1]/div/ul/li/div/a[1]/span/span').click() # 카풀 게시판 클릭
        elements = self.driver.find_elements_by_xpath('//*[@id="ct"]/div/ul/li') # 게시판 글 갯수 60개

        links = []  # 게시글 링크
        for element in elements:
            link = element.find_element_by_css_selector("a").get_attribute("href")
            links.append(link)
        return links
    
    def get_data(self):
        datas = []
        links = self.__get_list()
        for i in range(len(links)):
            try:
                self.driver.get(links[i])
                time.sleep(1)

                # 카풀 정보 가져오기
                dom = BeautifulSoup(self.driver.page_source , 'html.parser')
                title = dom.select_one('div.post_title > h2').text.strip()
                nick = dom.select_one('div.user_wrap > div.info > a.nick > span > span').text
                post_date = dom.select_one('div.user_wrap > div.info > span.date.font_l').text.split('작성일')[1]
                content = dom.select_one('#postContent').text.strip().split(' 투표는 표시되지 않습니다.')[0].replace("\n", "")
                time.sleep(1)

                # 작성자 정보 가져오기
                self.driver.find_element_by_xpath('//*[@id="ct"]/div[1]/div/div[2]/div[1]/a[1]/span').click()
                time.sleep(1)
                dom = BeautifulSoup(self.driver.page_source , 'html.parser')
                nick_info = dom.select_one('#memberInfo > div > div.info_profile > div > div.desc').text.split(' ')
                cafe_level = nick_info[0].split('등급')[1].split('방문')[0]
                cafe_visit = nick_info[1]
            except:
                pass

            print(i)
            datas.append({'title':title, 'nick':nick, 'level':cafe_level, 'visit':cafe_visit, 'post_date':post_date, 'content':content, 'link':links[i]})

        self.df = pd.DataFrame(datas)

        if not os.path.isfile('surfx.csv'): # if file does not exist 
            self.df.to_csv('surfx.csv', index=False, encoding='utf-8-sig')
        else: # else it exists so append without writing the header
            self.df.to_csv('surfx.csv', encoding='utf-8-sig', mode='a', header=False, index=False)
        print("CSV saved!")
    
    def get_carpool(self, date_data):
        self.date_data = date_data
        
        # csv 가져와서 중복데이터 삭제
        df = pd.read_csv('surfx.csv')
        df.drop_duplicates(subset=['title', 'nick'], keep='first', inplace=True) # 제목과 작성자 닉네임이 같을 경우 중복으로 여겨 제외시킴
        df.reset_index(inplace=True, drop=True)
        
        # [카풀제공] 데이터만 불러오기
        idx = []
        for i in range(len(df)):
            idx.append('[카풀제공]' in df['title'][i])
        surfx_df = df[idx]
        surfx_df.reset_index(inplace=True, drop=True)
    
        # 카풀 제공 날짜 찾기
        dates = []

        for i in range(len(surfx_df)):
            if '오늘' in surfx_df['title'][i]:
                p_date = surfx_df['post_date'][i].replace(".", "").split(" ")[0] # 작성일 불러오기
                p_date = datetime.datetime(year=int(p_date[:4]), month=int(p_date[4:6]), day=int(p_date[6:8])) # 불러온 작성일을 datetime형식에 넣어주기
                mmdd = p_date.strftime('%y%m%d') # 월일만 표시하기
            elif '내일' in surfx_df['title'][i]:
                p_date = surfx_df['post_date'][i].replace(".", "").split(" ")[0] # 작성일 불러오기
                p_date = datetime.datetime(year=int(p_date[:4]), month=int(p_date[4:6]), day=int(p_date[6:8])) # 불러온 작성일을 datetime형식에 넣어주기
                tomorrow = p_date + datetime.timedelta(days = 1) # 하루 더해주고
                tomorrow = tomorrow.strftime('%y%m%d') # 월일만 표시하기
                mmdd = tomorrow
            elif '내일' not in surfx_df['title'][i]:
                try: # 어떤 형식으로든 월도 함께 적어준 경우
                    d_data = re.search('[0-9]{1,2}일|/[0-9]{1,2}|\.[0-9]{1,2}', surfx_df['title'][i]).group() # 일 정보 불러오기
                    d_data = int(re.sub("[^0-9]", "", d_data)) # 숫자만 불러오기
                    m_data = re.search('[0-9]{1,2}월|[0-9]{1,2}/|[0-9]{1,2}\.', surfx_df['title'][i]).group() # 월 정보 불러오기
                    m_data = int(re.sub("[^0-9]", "", m_data)) # 숫자만 불러오기
                    mmdd = datetime.datetime(year=2020, month=m_data, day=d_data) # 찾아낸 월 & 일 정보를 datetime형식에 넣어주기
                    mmdd = mmdd.strftime('%y%m%d') # 월일만 표시하기
                except: #월은 생략하고 일만 적어준 경우
                    p_date = surfx_df['post_date'][i].replace(".", "").split(" ")[0] # 작성일 불러오기
                    p_date_day = int(p_date[6:8]) # 작성일 중 일 데이터만 불러오기
                    if p_date_day > d_data: # 작성일이 카풀일보다 큰 경우 예) 작성일: 9월27일, 카풀일: 10월1일
                        m_data = int(p_date[4:6]) + 1 # 카풀월은 작성월의 다음월이기 때문에 +1 해줌 
                    else: #작성일이 카풀일보다 작거나 같은 경우 예) 작성일: 9월3일, 카풀일: 9월5일
                        m_data = int(p_date[4:6]) # 카풀월은 작성월과 동일
                    mmdd = datetime.datetime(year=2020, month=m_data, day=d_data) # 찾아낸 월 & 일 정보를 datetime형식에 넣어주기
                    mmdd = mmdd.strftime('%y%m%d')
            else:
                mmdd = "???"

            dates.append(mmdd)
        
        surfx_df['carpool'] = dates # carpool 날짜 추가하기
        surfx_df = surfx_df[['carpool', 'title', 'nick', 'level', 'visit', 'post_date', 'content', 'link']] # 컬럼 순서 정리
        result = surfx_df[surfx_df['carpool'] == str(date_data)] # 검색한 날짜 데이터만 가져오기
        result.drop(columns=['carpool'], inplace=True)
        result.reset_index(inplace=True, drop=True)
        if result.empty:
            result = '검색하신 날짜에 카풀을 제공하지 않습니다'
        return result
        
    def close(self):
        self.driver.close()

Writing surfing/surfing/surfx/Surfx.py


In [6]:
!tree surfing

surfing
├── setup.py
└── surfing
    ├── __init__.py
    ├── surfx
    │   ├── Surfx.py
    │   └── __init__.py
    └── wsbfarm
        ├── Wsbfarm.py
        └── __init__.py

3 directories, 6 files


In [7]:
%%writefile surfing/surfing/__init__.py

from .wsbfarm.Wsbfarm import *  # .: 현재 디렉토리     .(상대경로) , /(절대경로)
from .surfx.Surfx import *

Overwriting surfing/surfing/__init__.py


In [8]:
%%writefile surfing/setup.py
from setuptools import setup, find_packages

setup(
    name = "surpool",
    version="0.0.1",
    author="dss14 JinWonGi",
    author_email="ansm204@icloud.com",
    packages=find_packages(),
    include_package_data=True,
)


Overwriting surfing/setup.py


In [9]:
!ls surfing

setup.py surfing


##### 2. package 설치

In [10]:
pip install -e surfing/

Obtaining file:///Users/stevenkim/Documents/project/03_crawling/surfing
  Running setup.py develop for surfing
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip list | grep sur

surfing                            0.0.1               /Users/stevenkim/Documents/project/03_crawling/surfing


In [12]:
!tree surfing

surfing
├── setup.py
├── surfing
│   ├── __init__.py
│   ├── surfx
│   │   ├── Surfx.py
│   │   └── __init__.py
│   └── wsbfarm
│       ├── Wsbfarm.py
│       └── __init__.py
└── surfing.egg-info
    ├── PKG-INFO
    ├── SOURCES.txt
    ├── dependency_links.txt
    └── top_level.txt

4 directories, 10 files


##### 3. package 구동

In [15]:
from surfing.surfing.wsbfarm import Wsbfarm as ws
from surfing.surfing.surfx import Surfx as sf

##### wsbfarm

In [16]:
wsb = ws.Wsbfarm()

In [17]:
wsb.beach_list()

,kor,eng
0,고성 송지호해변,SJH1
1,고성 천진해변,GCJ1
2,속초 속초해변,SCC1
3,양양 물치해변,MUL
4,양양 설악해변,YSK1
5,양양 기사문해변A,GSM1
6,양양 동산해변,DS1
7,양양 죽도해변A,JD1
8,양양 인구해변,IJ1
9,양양 갯마을해변,GMU1


In [18]:
wsb.login('wsb_email', 'wsb_pw')

Login Sucessful!!!


In [19]:
wsb.get_wave_beach('SJ1')

In [20]:
wsb.get_wave_data()

beach:SJH1 crawling ...
beach:GCJ1 crawling ...
beach:SCC1 crawling ...
beach:MUL crawling ...
beach:YSK1 crawling ...
beach:GSM1 crawling ...
beach:DS1 crawling ...
beach:JD1 crawling ...
beach:IJ1 crawling ...
beach:GMU1 crawling ...
beach:NA1 crawling ...
beach:YNA1 crawling ...
beach:GJMJ1 crawling ...
beach:SC1 crawling ...
beach:GJ1 crawling ...
beach:DJ1 crawling ...
beach:YWH1 crawling ...
beach:SHM1 crawling ...
beach:JH1 crawling ...
beach:SJ1 crawling ...
beach:SJ4 crawling ...
beach:DDP1 crawling ...
beach:MLP1 crawling ...
beach:JM1 crawling ...
beach:JM2 crawling ...
beach:JSG1 crawling ...
beach:JGJ1 crawling ...
beach:JIT1 crawling ...
beach:WJ1 crawling ...
beach:SKK1 crawling ...
WSBfarm CSV saved!


In [21]:
df = pd.read_csv('wsbfarm.csv')
df

,eng,date,summary,time,wind,temp,wave,wave_tide
0,SJH1,20200925,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,3.4m/s,18,2.4,0.13
1,SJH1,20200926,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 03,4.4m/s,18,2.8,0.21
2,SJH1,20200927,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오전 06,2.3m/s,15,2.5,0.28
3,SJH1,20200928,상급자 서핑하기 적당해요. 바다에서 해변으로 다소 약한 바람이 불며 성인 평균키 높...,오후 12,2.6m/s,19,1.7,0.36
4,SJH1,20200929,중급자 서핑하기 적당해요. 바다에서 해변으로 다소 약한 바람이 불며 허리~어깨 높이...,오전 06,1.7m/s,19,1.2,0.25
...,...,...,...,...,...,...,...,...
515,SKK1,20200930,역스웰에는 파도를 예측하기 힘들어요. 깜짝 파도를 대비해 주기적으로 파도웹캠을 확인...,오후 03,3.0m/s,24,0.5,0.75
516,SKK1,20201001,역스웰에는 파도를 예측하기 힘들어요. 깜짝 파도를 대비해 주기적으로 파도웹캠을 확인...,오전 09,1.9m/s,20,1.0,2.63
517,SKK1,20201002,역스웰에는 파도를 예측하기 힘들어요. 깜짝 파도를 대비해 주기적으로 파도웹캠을 확인...,오전 09,4.3m/s,21,0.5,2.70
518,SKK1,20201003,역스웰에는 파도를 예측하기 힘들어요. 깜짝 파도를 대비해 주기적으로 파도웹캠을 확인...,오전 09,1.6m/s,20,0.4,2.73


In [22]:
wsb.get_wave_date(20200925)

,beach,summary,time,wind,temp,max_wave,wave_tide
0,고성 송지호해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,3.4m/s,18,2.4,0.13
1,삼척 용화해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,3.2m/s,19,2.3,0.14
2,동해 대진해변A,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,2.6m/s,17,2.3,0.14
3,속초 속초해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,2.0m/s,18,2.2,0.13
4,고성 천진해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,2.0m/s,18,2.2,0.13
5,포항 신항만해변A,상급자 서핑 가능해요. 중급 및 상급자 서핑은 가능하지만 파도면이 좋지 않을 것으로...,오후 06,4.4m/s,20,2.1,0.30
6,양양 설악해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 03,2.6m/s,19,2.1,0.17
7,양양 물치해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 03,2.6m/s,19,2.1,0.17
8,강릉 사천해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,1.5m/s,17,2.0,0.14
9,강릉 주문진해변,상급자 이상 서핑 가능해요. 서핑 영화 '폭풍속으로'의 마지막 장면을 기억하시나요?...,오후 06,1.8m/s,17,1.9,0.13


In [23]:
wsb.close()

##### surfx

In [24]:
surfx = sf.Surfx()

In [25]:
# 로그인하기
surfx.login('naver_id', 'naver_pw')

Done


In [26]:
# 카풀 게시글 크롤링해서 csv로 저장
surfx.get_data()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
CSV saved!


In [27]:
pd.read_csv('surfx.csv')

,title,nick,level,visit,post_date,content,link
0,[카풀요청] 9/30(수) 서울 -> 양양 죽도 카풀요청합니다,계란말이케챱 서울,새내기써퍼,106,2020.09.25. 02:46,"​친구녀석 대신 글올립니다 ㅠ날짜를 다르게 따로따로 가게되었거든요...​뭇튼, 강남...",https://m.cafe.naver.com/ArticleRead.nhn?clubi...
1,[카풀요청] 9/26(토) 양양(죽도)>>>시흥웨이브파크(혹은 서울)오전출발차 구합니다!,최윤선 서울,새내기써퍼,"1,985",2020.09.24. 22:36,동호회 카플 가이드: http://cafe.naver.com/surfx/23647(...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
2,"[카풀제공] 9월 26일 토요일 강릉,양양-경기,서울 복귀 편도 카풀제공",몽상가들 안양,새내기써퍼,194,2020.09.24. 18:56,이번 주 토요일 양양에서 복귀편 카풀제공합니다 보드캐리 불가이구요 강릉이나 양양에서...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
3,[카풀제공] 9/26(토)~9/26(토) 서울 서울대입구역 주변 <-> 만리포 (당...,재훈 SEOUL,새내기써퍼,543,2020.09.24. 17:36,●목적지: 서울대입구역 주변 <-> 만리포 (편도가능)●서울출발장소: 서울대입구역 ...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
4,[카풀요청] 9/29 화요일 서울역->양양 카풀구해요!,에이미a 서울,새내기써퍼,"1,010",2020.09.24. 16:23,화요일 아침 7시이후 카풀요청드립니다!목적지는 죽도구요 흡연상관없고 보드없고 짐가방...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
5,[카풀제공] 9/25(금) 부천 -> 양양 죽도 -> 고성 편도 카풀 제공합니다. ...,물개 부천,새내기써퍼,"1,978",2020.09.24. 16:14,●언제: 9/25일 금요일 오전 6시 출발●목적지: 고성●만나는 장소와 시간: 부천...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
6,[카풀제공] 9/25일(금) 저녁 서울 → 양양 카풀제공합니다.,서대원 서울,새내기써퍼,66,2020.09.24. 15:34,●언제: 9/25일 금요일 오후 6시 출발 (서울 강서구 마곡나루역)●목적지: 양양...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
7,"[카풀제공] 9/26(토)저녁 7시이후 출발 강남 양재역→ 양양일대 ,주문진,강릉",공룡발 서울,새내기써퍼,"2,493",2020.09.24. 13:46,카플 가이드: http://cafe.naver.com/surfx/23647(위 가이...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
8,[카풀요청] 금진해변 -> 서울 강서구 화곡동 보드만 캐리해 주실분~~,리쉬 서울 화곡,새내기써퍼,"4,501",2020.09.24. 13:33,금진해변 -> 서울 강서구 화곡동 보드만 캐리해 주실분~~사례비는 얼마인지 몰라서 ...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
9,[카풀제공] 9/24(목)~9/26(토) 서울-양양,퉁췍 서울,새내기써퍼,361,2020.09.24. 13:08,목요일 2시 신대방 출발토요일 2시 양양 출발​경로상 픽업 가능비흡연안전 주행​공일...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...


In [30]:
# 날짜별 카풀 정보 찾기
surfx.get_carpool(200925)

,title,nick,level,visit,post_date,content,link
0,[카풀제공] 9/25(금) 부천 -> 양양 죽도 -> 고성 편도 카풀 제공합니다. ...,물개 부천,새내기써퍼,"1,978",2020.09.24. 16:14,●언제: 9/25일 금요일 오전 6시 출발●목적지: 고성●만나는 장소와 시간: 부천...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
1,[카풀제공] 9/25일(금) 저녁 서울 → 양양 카풀제공합니다.,서대원 서울,새내기써퍼,66,2020.09.24. 15:34,●언제: 9/25일 금요일 오후 6시 출발 (서울 강서구 마곡나루역)●목적지: 양양...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
2,[카풀제공] 25일(금)저녁 서울 → 양양 카풀 제공합니다.,버니타이거 서울,새내기써퍼,"3,814",2020.09.24. 09:40,※동호회 카플 가이드: http://cafe.naver.com/surfx/23647...,https://m.cafe.naver.com/ArticleRead.nhn?clubi...
3,"[카풀제공] 9/25(금) 가산(영등포,강남,잠실)->양양 카풀제공 합니다",까칠한 부천,새내기써퍼,"1,234",2020.09.24. 09:20,"●언제: 9/25(금) 가산디지털단지(영등포,강남,잠실등)->양양●목적지: 죽도(기...",https://m.cafe.naver.com/ArticleRead.nhn?clubi...


In [29]:
surfx.close()

##### 4. 패키지 삭제

In [31]:
!pip uninstall -y surfing

Found existing installation: surfing 0.0.1
Uninstalling surfing-0.0.1:
  Successfully uninstalled surfing-0.0.1


In [32]:
!pip list | grep sur